In [5]:

import numpy as np
import pandas as pd

def compute_hydro_firming(
    solar_mw: np.ndarray,
    dt_hours: float = 0.25,           # 15 minutes
    hydro_max_mw: float = np.inf,     # max hydro power (MW). Use np.inf if unconstrained.
    hydro_energy_mwh: float = np.inf, # daily hydro energy budget (MWh). Use np.inf if unconstrained.
    allow_curtailment: bool = True,
    tol: float = 1e-4,
    max_iter: int = 200
):
    """
    Compute hydro dispatch to firm solar to a flat output (load factor = 1) while respecting hydro constraints.
    
    Parameters
    ----------
    solar_mw : np.ndarray
        Solar forecast in MW for each 15-min interval (e.g., from 06:00 to 19:00 → 52 points).
    dt_hours : float
        Interval duration in hours (default 0.25 = 15 minutes).
    hydro_max_mw : float
        Hydro maximum instantaneous power (MW). Use np.inf if not limited.
    hydro_energy_mwh : float
        Hydro daily energy availability (MWh). Use np.inf if not limited.
    allow_curtailment : bool
        If True, solar can be curtailed so total delivered is exactly flat at C.
        If False, delivered = solar + hydro (may exceed C when solar is high).
    tol : float
        Tolerance for bisection on firm power C.
    max_iter : int
        Maximum iterations for bisection.
        
    Returns
    -------
    result : dict
        {
            "firm_power_mw": float,
            "hydro_mw": np.ndarray,
            "curtailment_mw": np.ndarray,
            "delivered_mw": np.ndarray,
            "feasible": bool,
            "used_hydro_energy_mwh": float
        }
    """
    solar = np.asarray(solar_mw, dtype=float)
    n = len(solar)
    if n == 0:
        raise ValueError("solar_mw must be a non-empty array")
    if hydro_max_mw < 0 or hydro_energy_mwh < 0:
        raise ValueError("hydro_max_mw and hydro_energy_mwh must be non-negative")

    # Helper: check feasibility for a candidate firm power C
    def feasible(C: float) -> bool:
        hydro_req = np.maximum(0.0, C - solar)
        if np.any(hydro_req - hydro_max_mw > 1e-12):  # instantaneous cap
            return False
        energy_need = hydro_req.sum() * dt_hours
        if energy_need - hydro_energy_mwh > 1e-9:     # daily energy cap
            return False
        return True

    # Upper bound for C is limited by instantaneous hydro cap:
    # At any interval, we can deliver at most solar + hydro_max_mw (with curtailment allowed).
    high_inst = np.min(solar + (0 if np.isinf(hydro_max_mw) else hydro_max_mw))

    # If no constraints, C could be arbitrarily large; but that's not physical.
    # We cap by instantaneous feasibility; energy feasibility is enforced via bisection.
    low, high = 0.0, high_inst

    # Edge case: if energy is severely limiting, high will be reduced by feasibility checks below.
    # Bisection to find the maximum feasible C
    for _ in range(max_iter):
        mid = 0.5 * (low + high)
        if feasible(mid):
            low = mid
        else:
            high = mid
        if (high - low) <= tol:
            break

    C = low
    # Build hydro schedule at C
    hydro = np.maximum(0.0, C - solar)
    # Enforce instantaneous cap explicitly (numerical guard)
    if not np.isinf(hydro_max_mw):
        hydro = np.minimum(hydro, hydro_max_mw)

    # Check energy budget; if slightly over due to tolerance, scale down hydro and reduce C accordingly
    used_energy = hydro.sum() * dt_hours
    if used_energy > hydro_energy_mwh + 1e-6:
        # scale hydro down proportionally to fit energy budget
        scale = (hydro_energy_mwh / used_energy) if used_energy > 0 else 0.0
        hydro *= scale
        # With less hydro, the actually deliverable flat output cannot be C without storage;
        # but if curtailment is allowed, we still target delivered=C by curtailing solar.
        # However, hydro < (C - solar)+ ⇒ delivered would drop below C at some times.
        # To keep LF=1 exactly, we recompute C to match updated hydro (one more bisection can be done).
        # Simple fix: recompute delivered strictly as min(C, solar + hydro) if you don't want a second bisection.
        # Here we re-run a quick one-shot refinement:
        def delivered_cap(Ctry):
            return np.all(solar + hydro + 1e-9 >= Ctry)
        # Reduce C to fit the (scaled) hydro
        low2, high2 = 0.0, min(C, np.min(solar + hydro))
        for _ in range(60):
            mid2 = 0.5 * (low2 + high2)
            if delivered_cap(mid2):
                low2 = mid2
            else:
                high2 = mid2
            if high2 - low2 <= tol:
                break
        C = low2

    # Curtailment if we require perfectly flat delivery at C
    if allow_curtailment:
        curtail = np.maximum(0.0, solar - C)
        delivered = np.full_like(solar, C)
    else:
        # No curtailment → actual delivered is solar + hydro (may exceed C)
        curtail = np.zeros_like(solar)
        delivered = solar + hydro

    return {
        "firm_power_mw": float(C),
        "hydro_mw": hydro,
        "curtailment_mw": curtail,
        "delivered_mw": delivered,
        "feasible": True,
        "used_hydro_energy_mwh": float((hydro * dt_hours).sum())
    }

In [7]:

# ---------- Example usage ----------
if __name__ == "__main__":
    # Example: 06:00 to 19:00, 15-min intervals → 52 points
    idx = pd.date_range("2026-01-16 06:00", "2026-01-16 19:00", freq="15min", inclusive="left")
    n = len(idx)

    # Create a smooth "bell-shaped" solar forecast (MW) peaking at noon
    x = np.linspace(-2.2, 2.2, n)
    solar_example = 80 * np.exp(-x**2)  # peak ~80 MW around noon

    # Define hydro constraints
    hydro_max_mw = 50.0          # max hydro power (MW)
    hydro_energy_mwh = 250.0     # daily hydro energy budget (MWh)
    dt_hours = 0.25              # 15 minutes

    result = compute_hydro_firming(
        solar_example,
        dt_hours=dt_hours,
        hydro_max_mw=hydro_max_mw,
        hydro_energy_mwh=hydro_energy_mwh,
        allow_curtailment=True
    )

    df = pd.DataFrame({
        "time": idx,
        "solar_mw": solar_example,
        "hydro_mw": result["hydro_mw"],
        "curtailment_mw": result["curtailment_mw"],
        "delivered_mw": result["delivered_mw"]
    })

    # Verify load factor (should be 1 when allow_curtailment=True)
    avg = df["delivered_mw"].mean()
    peak = df["delivered_mw"].max()
    load_factor = avg / peak if peak > 0 else np.nan

    print(f"Firm power (MW): {result['firm_power_mw']:.3f}")
    print(f"Used hydro energy (MWh): {result['used_hydro_energy_mwh']:.3f}")
    print(f"Load factor: {load_factor:.6f}")
    # df.to_csv("hydro_firm_schedule.csv", index=False)  # optional


Firm power (MW): 42.481
Used hydro energy (MWh): 249.999
Load factor: 1.000000
